# kaggle結果

In [2]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "w.jpg")

In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
import keras
from keras.backend import set_session
from keras.models import Sequential
from keras.layers import Dense , Dropout
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from pandas import DataFrame

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#設定GPU
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()

def create_session(gpu_id='0', pp_mem_frac=None):

    tf.reset_default_graph()
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id # can multiple?
    with tf.device('/gpu:' + gpu_id):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        if pp_mem_frac is not None:
            config.gpu_options.per_process_gpu_memory_fraction=pp_mem_frac
        session = tf.Session(config = config)
    return session
gpu_id = '0'
sess = create_session(gpu_id)
set_session(sess)
print("GPU Finished..............................")

GPU Finished..............................


# 資料前處理

In [3]:
#設定LSTM模型要往前預測幾步
timesteps = 6
print("------------------set constant---------------------")

------------------set constant---------------------


In [4]:
#刪掉雜訊
df = pd.read_csv("train.csv")
df = df[df["MISSING_DATA"] == False]
df = df[df["POLYLINE"] != "[]"]
df = df["POLYLINE"]
df.reset_index(drop=True, inplace=True)
print("df.shape :",df.shape)
print("---------------------remove train noise finish-----------------------------------")
dftest = pd.read_csv("test.csv")
dftest = dftest["POLYLINE"]
print("test_data : ",dftest.shape)
print("---------------------finish-----------------------------------")

df.shape : (1704759,)
---------------------remove train noise finish-----------------------------------
test_data :  (320,)
---------------------finish-----------------------------------


In [5]:
#資料前處理
df = df.str.replace("[","")
df = df.str.replace("]","")
dftest = dftest.str.replace("[","")
dftest = dftest.str.replace("]","")
print("---------------------split label and traindata-----------------------------------")
df = df.str.rsplit(",",2,True)
for i in range (df.shape[0]):
    if df.iloc[i][df.shape[1]-1] == None:#df.shape[1]-1 is because it's start from 0 
        df.iloc[i] = df.iloc[i].shift(1)
dflabel = df[[1,2]]
print("dflabel :",dflabel.shape)
dftrain = df[0]
print("---------------------finish-----------------------------------")

---------------------split label and traindata-----------------------------------
dflabel : (1704759, 2)
---------------------finish-----------------------------------


In [6]:
print("---------------------split traindata with timestep-----------------------------------")
#依照TIMESTEP分割資料並整理
dftrain = dftrain.str.rsplit(",",(timesteps*2),True)
dftest = dftest.str.rsplit(",",(timesteps*2),True)
for i in range (dftrain.shape[0]):
    if dftrain.iloc[i][dftrain.shape[1]-1] == None:#dftrain.shape[1]-1 is because it's start from 0 
        dftrain.iloc[i] = dftrain.iloc[i].shift(1)
        
dftrain = dftrain.iloc[:,1:]

---------------------split traindata with timestep-----------------------------------


In [8]:
#缺值補0
dftrain = dftrain.fillna(0)
dftrain

,1,2,3,4,5,6,7,8,9,10,11,12
0,-8.630838,41.15268,-8.632323,41.153022,-8.631144,41.154489,-8.630829,41.154507,-8.630829,41.154516,-8.630829,41.154498
1,-8.670852,41.165136,-8.670942,41.166576,-8.66961,41.167962,-8.668098,41.168988,-8.66664,41.170005,-8.665767,41.170635
2,-8.627814,41.144643,-8.625996,41.144769,-8.624088,41.144463,-8.621325,41.143401,-8.619444,41.141961,-8.617365,41.140863
3,-8.605854,41.142555,-8.607951,41.142753,-8.607978,41.142825,-8.607996,41.142879,-8.607987,41.142888,-8.608005,41.142915
4,-8.689059,41.17608,-8.689086,41.176431,-8.689059,41.176593,-8.687502,41.176755,-8.687259,41.177781,-8.687259,41.178078
5,-8.581626,41.153868,-8.582076,41.155434,-8.580546,41.155947,-8.578719,41.156271,-8.578629,41.157693,-8.578521,41.159439
6,-8.604945,41.144148,-8.603703,41.143608,-8.602236,41.142807,-8.603244,41.142681,-8.604,41.142798,-8.603982,41.142798
7,-8.624718,41.161563,-8.624727,41.161554,-8.624736,41.161545,-8.624736,41.161536,-8.624736,41.161536,-8.624727,41.161545
8,-8.596161,41.153679,-8.594838,41.155983,-8.594163,41.157135,-8.593002,41.159187,-8.591454,41.161608,-8.589924,41.163453
9,-8.607987,41.134518,-8.607861,41.134536,-8.60778,41.134545,-8.607411,41.134527,-8.605476,41.134392,-8.604603,41.134176


In [9]:
#把資料轉換成LSTM的格式
dflabel = dflabel.fillna(0)
dftrain = dftrain.fillna(0)
print("dftrain :",dftrain.shape)
dftrainnew = np.reshape(dftrain.values, (dftrain.shape[0], timesteps, 2))# reshape input to be [samples, time steps, features]
print("dftrain reshape:",dftrainnew.shape)
print("---------------------dftrain OK-----------------------------------")
for i in range (dftest.shape[0]):
    if dftest.iloc[i][dftest.shape[1]-1] == None:#dftest.shape[1]-1 is because it's start from 0 
        dftest.iloc[i] = dftest.iloc[i].shift(1)
dftest = dftest.iloc[:,1:]
dftest = dftest.fillna(0)
print("dftest :",dftest.shape)
dftestnew = np.reshape(dftest.values, (dftest.shape[0], timesteps, 2))# reshape input to be [samples, time steps, features]
print("dftest reshape:",dftestnew.shape)
print("---------------------dftest OK-----------------------------------")
print("---------------------finish-----------------------------------")


dftrain : (1704759, 12)
dftrain reshape: (1704759, 6, 2)
---------------------dftrain OK-----------------------------------
dftest : (320, 12)
dftest reshape: (320, 6, 2)
---------------------dftest OK-----------------------------------
---------------------finish-----------------------------------


# 開始訓練模型

In [11]:
history = LossHistory()

model = Sequential()
model.add(LSTM(256,input_shape=(timesteps,2), return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(128,input_shape = (timesteps,2), return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(32,input_shape = (timesteps,2)))
model.add(Dense(2,activation="linear"))
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, clipvalue=1.)
model.compile(loss='mean_squared_error', optimizer=sgd)
print("-------------model create finish------------------")

-------------model create finish------------------


In [12]:
model.fit(dftrainnew, dflabel,validation_split=0.1, epochs=50, verbose=2,batch_size = 500,callbacks=[history])

Train on 1534283 samples, validate on 170476 samples
Epoch 1/50
 - 180s - loss: 2.3807 - val_loss: 0.0019
Epoch 2/50
 - 112s - loss: 0.0030 - val_loss: 0.0018
Epoch 3/50
 - 109s - loss: 0.0030 - val_loss: 0.0019
Epoch 4/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 5/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 6/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 7/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 8/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 9/50
 - 109s - loss: 0.0030 - val_loss: 0.0019
Epoch 10/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 11/50
 - 109s - loss: 0.0030 - val_loss: 0.0019
Epoch 12/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 13/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 14/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 15/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 16/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 17/50
 - 110s - loss: 0.0030 - val_loss: 0.0019
Epoch 18/50
 - 110s - loss: 0.0030 - v

# 因為要上傳到kaggle所以要輸出檔案

In [13]:

#dftnew = np.reshape(dft.values, (dft.shape[0], int(len(dft.iloc[0])/2), 2))
output = model.predict(dftestnew)
output = DataFrame(output)
output = output[[1,0]]
output.columns = ['LATITUDE','LONGITUDE']

In [15]:
output.to_csv("output128_64_32_drop02_useclipvalue_timestep_4.csv")

# 心得
這次的kaggle比賽我嘗試了很多一些模型，包含ann ,logistic regression ,linear regression等等，然而這些模型都無法了解關於計程車路徑資料會有的時間性的，因此後來決定用lstm這個專門處理時間訊列的模型跑跑看，而中間也發生很多問題，最麻煩的地方就是資料前處理了，因為lstm模型要的資料是3維的所以剛接觸時不知道該怎麼分割，所幸後來有找到方法，也得出了比ann那些模型更好的結果